# PoC-boardgamegeeks

Use the "Run" button to execute the code.

In [ ]:
!pip install opendatasets --upgrade --quiet
!pip install pandas --upgrade --quiet
!pip install pandas-profiling


In [ ]:
import opendatasets as od
import os
import pandas as pd
import numpy as np
import psycopg2.extras as extras
from pandas_profiling import ProfileReport

### Retrieving Dataset from Kaggle 

In [ ]:
#Setting a variable for the kaggle dataset link
dataset_url = 'https://www.kaggle.com/threnjen/board-games-database-from-boardgamegeek' 


#### - You will need a kaggle account and a kaggle key in order to download this data set
#### - For the kaggle key, log in to your kaggle account page, scroll down to API - Create New API Token.
#### - Download and open the file and your kaggle key will be there.

In [ ]:
od.download(dataset_url)

In [ ]:
#Setting a variable to the path of the dataset folder
data_dir = './board-games-database-from-boardgamegeek'

In [ ]:
#Listing the contents of the path directory to make sure we have all the cvs files
os.listdir(data_dir)

In [ ]:
pd.set_option('display.max_columns', None) #Always show all columns
pd.set_option('display.max_rows', 10) #Always show upto 200 rows

In [ ]:
games_raw_df=pd.read_csv('./board-games-database-from-boardgamegeek/games.csv')


### Examine the column names of the dataset

In [ ]:
games_raw_df.columns


In [ ]:
games_raw_df.columns = ['BGGId', 'Name', 'Description', 'YearPublished', 'GameWeight',
       'AvgRating', 'BayesAvgRating', 'StdDev', 'MinPlayers', 'MaxPlayers',
       'ComAgeRec', 'LanguageEase', 'BestPlayers', 'GoodPlayers', 'NumOwned',
       'NumWant', 'NumWish', 'NumWeightVotes', 'MfgPlaytime', 'ComMinPlaytime',
       'ComMaxPlaytime', 'MfgAgeRec', 'NumUserRatings', 'NumComments',
       'NumAlternates', 'NumExpansions', 'NumImplementations',
       'IsReimplementation', 'Family', 'Kickstarted', 'ImagePath',
       'RankBoardgame', 'RankStrategygames', 'RankAbstracts',
       'RankFamilygames', 'RankThematic', 'RankCgs', 'RankWargames',
       'RankPartygames', 'RankChildrensgames', 'CatThematic',
       'CatStrategy', 'CatWar', 'CatFamily', 'CatCGS', 'CatAbstract',
       'CatParty', 'CatChildrens']

In [ ]:
games_raw_df.describe()

##### Dropping unwanted columns

In [ ]:
games_updated_df = games_raw_df.drop(['StdDev',"ComAgeRec","LanguageEase","GameWeight","BayesAvgRating",'NumOwned',
       'NumWant', 'NumWish', 'NumWeightVotes', 'NumComments',
       'NumAlternates', 'NumExpansions', 'NumImplementations',
       'IsReimplementation', 'Family', 'Kickstarted', 'ImagePath'], axis = 1) 


In [ ]:
games_updated_df1 = games_updated_df.round({"AvgRating":2}) 

In [ ]:
import psycopg2

#establish client connection - specify param values
conn = psycopg2.connect("host=localhost dbname=bgg user=pocuser password=poc123")

#cursor facilitates processing and executing commands such as traverse, add, remove, retrieve in a database
cur = conn.cursor()

#cur.execute('SELECT * FROM notes')
# one = cur.fetchone()
# all = cur.fetchall()


### Create Tables


#### games_flat columns and descriptions
    BGGId			BoardGameGeek game ID
    Name			Name of game
    YearPublished	First year game published
    Category		Category the game falls under
    MfgPlayTime	Manufacturer Stated Play Time
    MinPlayers		Minimum number of players
    MaxPlayers		Maximun number of players
    AvgRatings		Average user rating for game
    MfgAgeRecs		Manufacturer Age Recommendation
    NumUserRatingss	Number of user ratings


In [ ]:
#Create the games flat table

import psycopg2
conn = psycopg2.connect("host=localhost dbname=bgg user=pocuser password=poc123")
cur = conn.cursor()
cur.execute("""
    CREATE TABLE games_flat(
        bggid INTEGER PRIMARY KEY,
        name TEXT,
        yearpublished INTEGER,
        category TEXT,
        mfgplaytime INTEGER,
        minplayers INTEGER,
        maxplayers INTEGER,
        avgrating REAL,
        mfgagerec INTEGER,
        numuserratings INTEGER
)
""")
conn.commit()

In [ ]:
#Create the games table

import psycopg2
conn = psycopg2.connect("host=localhost dbname=bgg user=pocuser password=poc123")
cur = conn.cursor()
cur.execute("""
    CREATE TABLE games(
        bggid INTEGER PRIMARY KEY,
        name TEXT,
        yearpublished INTEGER,
        mfgagerec INTEGER
)
""")
conn.commit()

In [ ]:
#Create the players table

import psycopg2
conn = psycopg2.connect("host=localhost dbname=bgg user=pocuser password=poc123")
cur = conn.cursor()
cur.execute("""
    CREATE TABLE users(
        bggid INTEGER PRIMARY KEY,
        minplayers INTEGER,
        maxplayers INTEGER
        )
""")
conn.commit()

In [ ]:
#Create the ratings table

import psycopg2
conn = psycopg2.connect("host=localhost dbname=bgg user=pocuser password=poc123")
cur = conn.cursor()
cur.execute("""
    CREATE TABLE ratings (
        bggid INTEGER PRIMARY KEY,
        avgrating INTEGER,
        numuserratings INTEGER
)
""")
conn.commit()



In [ ]:
#Create the playtime table

import psycopg2
conn = psycopg2.connect("host=localhost dbname=bgg user=pocuser password=poc123")
cur = conn.cursor()
cur.execute("""
    CREATE TABLE playtime(
        BGGId INTEGER PRIMARY KEY,
        MfgPlaytime INTEGER
)
""")
conn.commit()

In [ ]:
#Create the playtime table

import psycopg2
conn = psycopg2.connect("host=localhost dbname=bgg user=pocuser password=poc123")
cur = conn.cursor()
cur.execute("""
    CREATE TABLE category(
        BGGId INTEGER PRIMARY KEY,
        category text
)
""")
conn.commit()

## select_sql() and execute_values() functions

In [ ]:
#defining a function to read the sql query and print out into jupyter notebook

def select_sql(sql):
    return pd.read_sql(sql, conn)

In [ ]:
#Defining a funtion to export the dataframe from jupter into the postgres database - tables

def execute_values(conn, df, table):

    #Using psycopg2.extras.execute_values() to insert the dataframe

    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    
    # SQL query to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

## Games filtering for creating smaller dataframes

In [ ]:
games_updated_df2 = games_updated_df1.filter(['BGGId'
                                              ,'Name'
                                              #,'Description'
                                              #,'YearPublished'
                                              #,'AvgRating'
                                              #,'MinPLayer'
                                              #,'MaxPlayers'
                                              #,'BestPlayers'
                                              #,'MfgPlaytime'
                                              #,"ComMinPlaytime"
                                              #,'ComMaxPLaytime'
                                              #,'MfgAgeRec'
                                              #,'NumUserRatings'
                                              #,'RankBoardgame'
                                              #,'RankStrategygames'
                                              #,'RankAbstracts'
                                              #,'RankFamilygames'
                                              #,'RankThematic'
                                              #,'RankCgs'
                                              #,'RankWargames'
                                              #,'RankPartygames'
                                              #,'RankChildrensgames'
                                              ,'CatThematic'
                                              ,'CatStrategy'
                                              , 'CatWar'
                                              , 'CatFamily'
                                              , 'CatCGS'
                                              , 'CatAbstract'
                                              ,'CatParty'
                                              , 'CatChildrens'
                                             ])

In [ ]:
games_updated_df2

In [ ]:
def from_one_hot_to_category(row, names_array): #take a row of a dataframe and a array of names
    cat_array = np.array(row) #interpret the row as an np.array
    names_array = [" " + s + "," for s in names_array] #Add an " " at the beginning and a "," at the end of every name in names_array
    return ''.join(np.repeat(names_array, cat_array)) #Output names_array * cat_array.transpose, whereby string * int = string if int = 1 and 0 else

In [ ]:
category_df = games_updated_df2
games_updated_df2['Category'] = games_updated_df2[['CatThematic',
       'CatStrategy', 'CatWar', 'CatFamily', 'CatCGS', 'CatAbstract',
       'CatParty', 'CatChildrens']].apply(lambda row: from_one_hot_to_category(row, list(row.index)), axis = 1) #apply from_one_hot_to_category rowwise

games_updated_df2['Category'] = games_updated_df2['Category'].str[1:-1].str.replace('Cat','') #Get rid of the first " " at left hand side and the last "," at the right hand side

category_df = category_df.merge(games_updated_df2[['BGGId', 'Category']], on = 'BGGId', how = 'left') #Add the new column Category to sub_df

In [ ]:
category_df

In [ ]:
category_df = category_df.drop(['Name','CatThematic', 'CatStrategy', 'CatWar','CatFamily', 'CatAbstract',"CatCGS",'CatParty','CatChildrens', "Category_y"], axis = 1)

In [ ]:
#df.rename(columns={"A": "a", "B": "c"})
category_df=category_df.rename(columns={"Category_x": "Category"})
category_df

In [ ]:
category_df.Category.unique()

### As you can see the categories has a blank category as well as more than one for some, we will clean this up to only look at games with one category only

In [ ]:
category_df.replace('', np.nan, inplace=True)

category_df.dropna(subset = ['Category'], inplace=True)


In [ ]:
category_df.Category.unique()

In [ ]:
#This options variable is used to select on the one category types

options = ['Thematic'
           ,'Strategy'
           , 'War'
           , 'Family'
           , 'CGS'
           , 'Abstract'
           ,'Party'
           , 'Childrens']
# selecting rows based on condition
category_cleaned_df = category_df[category_df['Category'].isin(options)]

In [ ]:
category_cleaned_df.Category.unique()

In [ ]:
games_id_name_year_df = games_updated_df1.filter(['BGGId', 'Name', 'YearPublished'], axis = 1)

### Remove any anomalies in the table with the yearpublished as 0. This could have been a place holder.

In [ ]:
games_id_name_year_df.replace(0, np.nan, inplace=True)

games_id_name_year_df.dropna(subset = ['YearPublished'], inplace=True)

In [ ]:
games_id_name_year_df

### Convert the year datatype as there seems to be a decimal 0 at the end

In [ ]:
games_id_name_year_df = games_id_name_year_df.convert_dtypes()
games_id_name_year_df

### Now to merge with the category_cleaned_df

In [ ]:
#df = pd.merge(df1, df2, on="ID")
games_id_name_year_cat_df = pd.merge(games_id_name_year_df, category_cleaned_df, on="BGGId",  how="inner")

In [ ]:
games_id_name_year_cat_df

### Build the playtime_df in order to clean (remove large play times of 0 and greater than 210 minutes) and merge

In [ ]:
games_id_playtime_df = games_updated_df1.filter(['BGGId', 'MfgPlaytime'], axis = 1)

In [ ]:
games_id_playtime_df.replace(0, np.nan, inplace=True)

games_id_playtime_df.dropna(subset = ['MfgPlaytime'], inplace=True)

In [ ]:
games_id_playtime_df2 = games_id_playtime_df[games_id_playtime_df['MfgPlaytime']<210]

In [ ]:
games_id_playtime_df2

In [ ]:
games_id_name_cat_year_playtime_df = pd.merge(games_id_name_year_cat_df, games_id_playtime_df2, on="BGGId",  how="inner")

In [ ]:
games_id_name_cat_year_playtime_df

In [ ]:
games_id_minplayers_df = games_updated_df1.filter(['BGGId', 'MinPlayers'], axis = 1)

games_id_minplayers_df.replace(0, np.nan, inplace=True)

games_id_minplayers_df.dropna(subset = ['MinPlayers'], inplace=True)

games_incypm_df = pd.merge(games_id_name_cat_year_playtime_df, games_id_minplayers_df, on="BGGId",  how="inner")

games_incypm_df

In [ ]:
games_id_maxplayers_df = games_updated_df1.filter(['BGGId', 'MaxPlayers'], axis = 1)

games_id_maxplayers_df.replace(0, np.nan, inplace=True)

games_id_maxplayers_df.dropna(subset = ['MaxPlayers'], inplace=True)

games_incypmx_df = pd.merge(games_incypm_df, games_id_maxplayers_df, on="BGGId",  how="inner")

games_incypmx2_df = games_incypmx_df[games_incypmx_df['MaxPlayers']<21]

games_incypmx2_df

In [ ]:
games_id_ageuser_df = games_updated_df1.filter(['BGGId', 'AvgRating', 'MfgAgeRec', 'NumUserRatings'], axis = 1)

games_id_ageuser_df.replace(0, np.nan, inplace=True)

games_id_ageuser_df.dropna(subset = ['AvgRating', 'MfgAgeRec','NumUserRatings'], inplace=True)

games_cleaner_df = pd.merge(games_incypmx2_df, games_id_ageuser_df, on="BGGId",  how="inner")

games_cleaner_df

### Here is the flattened and cleaned table which will be used to do some analysis

In [ ]:
games_cleaner_df = games_cleaner_df.convert_dtypes()
games_cleaner_df

### NOW, we can export these dataframes into the postgres tables using the function execute_values defined above

In [ ]:
execute_values(conn, games_cleaner_df, 'games_flat')

## Do a little profiling with pandas-profiling. Find the report_games.html in your directory or you can view the one in tthe github repository.

In [ ]:
profile = ProfileReport(games_cleaner_df, title = 'Games Flattened and Cleaned', explorative= True)
#profile.to_notebook_iframe()
profile.to_file('report_games.html')

### Now lets make sure it has properly loaded into the database. If it has, we can retrieve it from inside this norebook using the select_sql function defined earlier! Don't mind the warning popping up, we just need to see if it works or not.

In [ ]:
sql = '''select * from games_flat limit 10'''
select_sql(sql)

### Now lets chop this cleaned up flat dataframe into smaller dataframes for exporting into the seperate tables

In [ ]:
games_df = games_cleaner_df.filter(['BGGId', 'Name', 'YearPublished', 'MfgAgeRec'])
users_df = games_cleaner_df.filter(['BGGId', 'MinPlayers', 'MaxPlayers'])
ratings_df = games_cleaner_df.filter(['BGGId', 'AvgRating', 'NumUserRatings'])
playtime_df = games_cleaner_df.filter(['BGGId', 'MfgPlaytime'])
category_df = games_cleaner_df.filter(['BGGId', 'Category'])


In [ ]:
execute_values(conn, games_df, 'games')

In [ ]:
execute_values(conn, users_df, 'users')

In [ ]:
execute_values(conn, ratings_df, 'ratings')

In [ ]:
execute_values(conn, playtime_df, 'playtime')

In [ ]:
execute_values(conn, category_df, 'category')

### Let's Query each one just to be sure

In [ ]:
sql = '''select * from games limit 10'''
select_sql(sql)

In [ ]:
sql = '''select * from users limit 10'''
select_sql(sql)

In [ ]:
sql = '''select * from playtime limit 10'''
select_sql(sql)

In [ ]:
sql = '''select * from category limit 10'''
select_sql(sql)

In [ ]:
import psycopg2
#establish connection
conn = psycopg2.connect("host=localhost dbname=bgg user=pocuser password=poc123")
#cursor facilitates processing such as traverse, add, remove, retrieve in a database
cur = conn.cursor()
cur.execute('SELECT * FROM ratings')

one = cur.fetchone()
all = cur.fetchall()

In [ ]:
one